# Short-Circuit Calculation according to IEC 60909

pandapower supports short-circuit calculations with the method of equivalent voltage source at the fault location according to IEC 60909. The pandapower short-circuit calculation supports the following elements:

- sgen (as motor or as full converter generator or as asynchronous machine or as doubly-fed asynchronous machine)
- gen (as synchronous generator)
- ext_grid
- line
- trafo
- trafo3w
- impedance

with the correction factors as defined in IEC 60909. Loads and shunts are neglected as per standard. The pandapower switch model is fully integrated into the short-circuit calculation. 

The following short-circuit currents can be calculated:
- ikss (Initial symmetrical short-circuit current)
- ip (short-circuit current peak)
- ith (equivalent thermal short-circuit current)

either as

 - symmetrical three-phase or
 - asymmetrical two-phase
 
short circuit current. Calculations are available for meshed as well as for radial networks. ip and ith are only implemented for short circuits far from synchronous generators.

In addition, short-circuit calculation with the superposition method is implemented. With this method, the pre-fault voltage is considered. The following differences to the worst-case calculation apply:
- transformer correction factor is not applied
- load, sgen are additionally modelled as shunt admittances (calculated based on their pre-fault currents)
- the grid must contain the results of a successful power flow calculation

The results for all elements and different short-circuit currents are tested against commercial software to ensure that correction factors are correctly applied. 

### Example Network

Here is a little example on how to use the short-circuit calculation. First, we create a simple open ring network with 4 buses, that are connected by one transformer and two lines with one open sectioning point. The network is fed by an external grid connection at bus 1:

<img src="shortcircuit/example_network_sc.png">

In [1]:
import pandapower as pp
import pandapower.shortcircuit as sc

def ring_network():
    net = pp.create_empty_network()
    b1 = pp.create_bus(net, 220)
    b2 = pp.create_bus(net, 110)
    b3 = pp.create_bus(net, 110)
    b4 = pp.create_bus(net, 110)
    pp.create_ext_grid(net, b1, s_sc_max_mva=100., s_sc_min_mva=80., rx_min=0.20, rx_max=0.35)
    pp.create_transformer(net, b1, b2, "100 MVA 220/110 kV")
    pp.create_line(net, b2, b3, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=15.)
    l2 = pp.create_line(net, b3, b4, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=12.)
    pp.create_line(net, b4, b2, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=10.)
    pp.create_switch(net, b4, l2, closed=False, et="l")
    return net

## Symmetric Short-Circuit Calculation

### Maximum Short Circuit Currents
Now, we load the network and calculate the maximum short-circuit currents with the calc_sc function:

In [2]:
net = ring_network()
sc.calc_sc(net, case="max", ip=True, ith=True, branch_results=True)
net.res_bus_sc

hp.pandapower.shortcircuit.calc_sc - WARNING: Branch results are in beta mode and might not always be reliable, especially for transformers
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:240: RuntimeWarning: invalid value encountered in log
  m = (np.exp(4 * f * tk_s * np.log(kappa - 1)) - 1) / (2 * f * tk_s * np.log(kappa - 1))
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:654: RuntimeWarning: All-NaN slice encountered
  ppci["branch"][:, IP_F] = np.nanmax(np.abs(ip_all_f), axis=1) / baseI[fb]
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:655: RuntimeWarning: All-NaN slice encountered
  ppci["branch"][:, IP_T] = np.nanmax(np.abs(ip_all_t), axis=1) / baseI[tb]
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:671: RuntimeWarning: All-NaN slice encountered
  ppci["branch"][:, ITH_F] = np.nanmax(np.abs(ith_all_f), axis=1) / baseI[fb]
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircu

,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.262432,100.000000,0.505834,0.263723,175.878566,502.510189
1,0.476454,90.776637,0.942589,0.479039,44.276323,139.778739
2,0.466671,88.912758,0.915418,0.469123,46.571323,142.268739
3,0.469892,89.526397,0.924301,0.472386,45.806323,141.438739


where ikss is the initial short-circuit current, ip is the peak short-circuit current and ith is the thermal equivalent current.

For branches, the results are defined as the maximum current flows through that occurs for a fault at any bus in the network. The results are available seperately for lines:

In [3]:
net.res_line_sc

,ikss_ka,ikss_from_ka,ikss_from_degree,ikss_to_ka,ikss_to_degree,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,ip_ka,ith_ka
0,0.466671,0.466671,-71.874268,0.466671,108.125732,1.499428,1.62683,0.000000,0.000000,0.024883,-24.540621,0.016703,-24.721299,0.915418,0.469123
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.016703,-24.721299,0.016703,-24.721299,0.000000,0.000000
2,0.469892,0.469892,107.945055,0.469892,-72.054945,0.000000,0.00000,1.013464,1.099575,0.024883,-24.540621,0.024883,-24.540621,0.924301,0.472386


and transformers:

In [4]:
net.res_trafo_sc

,ikss_hv_ka,ikss_hv_degree,ikss_lv_ka,ikss_lv_degree,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree
0,0.238227,-72.423878,0.476454,107.576122,1.699796,10.872445,0.0,0.0,0.123768,9.240049,0.024883,-24.540621


### Minimum Short Circuit Currents

Minimum short-circuits can be calculated in the same way. However, we need to specify the end temperature of the lines after a fault as per standard first:

In [5]:
net = ring_network()
net.line["endtemp_degree"] = 80
sc.calc_sc(net, case="min", ith=True, ip=True, branch_results=True)
net.res_bus_sc

hp.pandapower.shortcircuit.calc_sc - WARNING: Branch results are in beta mode and might not always be reliable, especially for transformers
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:240: RuntimeWarning: invalid value encountered in log
  m = (np.exp(4 * f * tk_s * np.log(kappa - 1)) - 1) / (2 * f * tk_s * np.log(kappa - 1))
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:654: RuntimeWarning: All-NaN slice encountered
  ppci["branch"][:, IP_F] = np.nanmax(np.abs(ip_all_f), axis=1) / baseI[fb]
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:655: RuntimeWarning: All-NaN slice encountered
  ppci["branch"][:, IP_T] = np.nanmax(np.abs(ip_all_t), axis=1) / baseI[tb]
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:671: RuntimeWarning: All-NaN slice encountered
  ppci["branch"][:, ITH_F] = np.nanmax(np.abs(ith_all_f), axis=1) / baseI[fb]
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircu

,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.209946,80.000000,0.462534,0.211736,118.650262,593.251309
1,0.384422,73.242307,0.860874,0.387974,29.969247,162.464019
2,0.377608,71.943998,0.832832,0.380846,32.815047,164.954019
3,0.379861,72.373180,0.841982,0.383197,31.866447,164.124019


The branch results are now the minimum current flows through each branch:

In [6]:
net.res_line_sc

,ikss_ka,ikss_from_ka,ikss_from_degree,ikss_to_ka,ikss_to_degree,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,ip_ka,ith_ka
0,0.377608,0.377608,-78.748780,0.377608,101.251220,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,8.328324e-01,3.808460e-01
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,1.000000e+10,1.000000e+10
2,0.379861,0.379861,100.987879,0.379861,-79.012121,0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,8.419815e-01,3.831970e-01


In [7]:
net.res_trafo_sc

,ikss_hv_ka,ikss_hv_degree,ikss_lv_ka,ikss_lv_degree,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree
0,0.188804,-78.74878,0.377608,101.25122,0.0,0.0,-1.217329,-1.065131,0.0,0.0,0.0,0.0


### Asynchronous Motors

Asynchronous motors can be specified by creating a static generator of type "motor". For the short circuit impedance, an R/X ratio "rx" as well as the ratio between nominal current and short circuit current "k" has to be specified:

In [8]:
net = ring_network()
pp.create_sgen(net, 2, p_mw=0, sn_mva=0.5, k=1.2, rx=7., type="motor")
net

This pandapower network includes the following parameter tables:
   - bus (4 element)
   - sgen (1 elements)
   - switch (1 elements)
   - ext_grid (1 elements)
   - line (3 element)
   - trafo (1 elements)

If we run the short-circuit calculation again, we can see that the currents increased due to the contribution of the inverteres to the short-circuit currents.

In [9]:
sc.calc_sc(net, case="max", ith=True, ip=True)
net.res_bus_sc

c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:240: RuntimeWarning: invalid value encountered in log
  m = (np.exp(4 * f * tk_s * np.log(kappa - 1)) - 1) / (2 * f * tk_s * np.log(kappa - 1))


,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.264007,100.600000,0.508061,0.265306,175.878566,502.510189
1,0.479603,91.376637,0.947043,0.482205,44.276323,139.778739
2,0.469820,89.512758,0.919871,0.472289,46.571323,142.268739
3,0.472998,90.118134,0.928693,0.475509,45.806323,141.438739


### Synchronous Generators

Synchronous generators can also be considered in the short-circuit calculation with the gen element. According to the standard, the rated power factor (cos$\varphi$) "cos_phi", rated voltage "vn_kv", rated apparent power "sn_kva" and subtransient resistance "rdss" and reactance "xdss" are necessary to calculate the short circuit impedance:

In [10]:
net = ring_network()
pp.create_gen(net, 2, p_mw=0, vm_pu=1.0, cos_phi=0.8, vn_kv=22, sn_mva=5, xdss_pu=0.2, rdss_ohm=0.005)
net

This pandapower network includes the following parameter tables:
   - bus (4 element)
   - gen (1 elements)
   - switch (1 elements)
   - ext_grid (1 elements)
   - line (3 element)
   - trafo (1 elements)

and run the short-circuit calculation again:

In [11]:
sc.calc_sc(net, case="max", ith=True, ip=True)
net.res_bus_sc

hp.pandapower.shortcircuit.calc_sc - WARNING: aperiodic, thermal short-circuit currents are only implemented for faults far from generators!
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:240: RuntimeWarning: invalid value encountered in log
  m = (np.exp(4 * f * tk_s * np.log(kappa - 1)) - 1) / (2 * f * tk_s * np.log(kappa - 1))


,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.568039,216.451899,1.285305,0.573572,40.720778,242.572728
1,1.180874,224.986699,2.710856,1.193297,8.060258,58.607387
2,1.187256,226.202637,2.777175,1.201155,7.204747,58.398291
3,1.144754,218.104981,2.570526,1.155476,9.590258,60.267387


Once again, the short-circuit current increases due to the contribution of the generator. As can be seen in the warning, the values for peak and thermal equivalent short-circuit current will only be accurate for faults far from generators.

## Meshed Networks

The correction factors for aperiodic and thermal currents differ between meshed and radial networks. pandapower includes a meshing detection that automatically detects the meshing for each short-circuit location. Alternatively, the topology can be set to "radial" or "meshed" to circumvent the check and save calculation time.

We load the radial network and close the open sectioning point to get a closed ring network:

In [12]:
net = ring_network()
net.switch.closed = True
sc.calc_sc(net, topology="auto", ip=True, ith=True)
net.res_bus_sc

,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.262432,100.000000,0.505834,0.263723,175.878566,502.510189
1,0.476454,90.776637,0.942589,0.479039,44.276323,139.778739
2,0.470593,89.660073,0.926244,0.473098,45.640917,141.259279
3,0.471649,89.861256,0.929174,0.474168,45.392809,140.990090


the network is automatically detected to be meshed and application factors are applied. This can be validated by setting the topology to radial and comparing the results:

In [13]:
sc.calc_sc(net, topology="radial", ip=True, ith=True)
net.res_bus_sc

,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.262432,100.000000,0.505834,0.263723,175.878566,502.510189
1,0.476454,90.776637,0.942589,0.479039,44.276323,139.778739
2,0.470593,89.660073,0.926244,0.473098,45.640917,141.259279
3,0.471649,89.861256,0.929174,0.474168,45.392809,140.990090


If we look at the line results, we can see that the line currents are significantly smaller than the bus currents:

In [14]:
sc.calc_sc(net, topology="auto", ip=True, ith=True, branch_results=True)
net.res_line_sc

hp.pandapower.shortcircuit.calc_sc - WARNING: Branch results are in beta mode and might not always be reliable, especially for transformers


,ikss_ka,ikss_from_ka,ikss_from_degree,ikss_to_ka,ikss_to_degree,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,ip_ka,ith_ka
0,0.279812,0.279812,-72.094339,0.279812,107.905661,0.539060,0.584863,0.000000,0.000000,0.014920,-24.760693,0.005438,-24.820004,0.550740,0.281301
1,0.190781,0.190781,107.905661,0.190781,-72.094339,0.089501,0.097106,0.200477,0.217511,0.005438,-24.820004,0.008138,-24.760693,0.375504,0.191796
2,0.344176,0.344176,107.846349,0.344176,-72.153651,0.000000,0.000000,0.543720,0.589918,0.008138,-24.760693,0.014920,-24.760693,0.678046,0.346014


this is because the short-circuit current is split up on both paths of the ring, which is correctly considered by pandapower.

## Fault Impedance

It is also possible to specify a fault impedance in the short-circuit calculation:

In [15]:
net = ring_network()
sc.calc_sc(net, topology="radial", ip=True, ith=True, r_fault_ohm=1., x_fault_ohm=2.)

c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\kappa.py:25: RuntimeWarning: invalid value encountered in divide
  kappa = _kappa(ppc["bus"][:, R_EQUIV] / ppc["bus"][:, X_EQUIV])
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:240: RuntimeWarning: invalid value encountered in log
  m = (np.exp(4 * f * tk_s * np.log(kappa - 1)) - 1) / (2 * f * tk_s * np.log(kappa - 1))


which of course decreases the short-circuit currents:

In [16]:
net.res_bus_sc

,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.261343,99.585098,0.503509,0.262627,176.878566,504.510189
1,0.469382,89.429305,0.926656,0.471909,45.276323,141.778739
2,0.459875,87.617950,0.900379,0.462274,47.571323,144.268739
3,0.463005,88.214388,0.908972,0.465446,46.806323,143.438739


## Superposition method

We need to first execute the power flow calculation so that the grid has valid power flow results. We rely on the user to execute this step explicitly so that the user is fully aware that a power flow calculation is executed and also has control over all the relevant options for the power flow calculation.

Next, we pass the parameter use\_pre\_fault\_voltage = True to use the superposition method.

In [17]:
pp.runpp(net)
sc.calc_sc(net, topology="radial", ip=True, ith=True, r_fault_ohm=1., x_fault_ohm=2., 
           branch_results=True, return_all_currents=True)

hp.pandapower.shortcircuit.calc_sc - WARNING: Branch results are in beta mode and might not always be reliable, especially for transformers
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\kappa.py:25: RuntimeWarning: invalid value encountered in divide
  kappa = _kappa(ppc["bus"][:, R_EQUIV] / ppc["bus"][:, X_EQUIV])
c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:240: RuntimeWarning: invalid value encountered in log
  m = (np.exp(4 * f * tk_s * np.log(kappa - 1)) - 1) / (2 * f * tk_s * np.log(kappa - 1))


In [18]:
net.res_bus_sc

,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.261343,99.585098,0.503509,0.262627,176.878566,504.510189
1,0.469382,89.429305,0.926656,0.471909,45.276323,141.778739
2,0.459875,87.617950,0.900379,0.462274,47.571323,144.268739
3,0.463005,88.214388,0.908972,0.465446,46.806323,143.438739


In [19]:
net.res_line_sc

ikss_ka  ikss_from_ka  ikss_from_degree  ikss_to_ka  \
line bus                                                         
0    0    0.000000      0.000000          0.000000    0.000000   
     1    0.000000      0.000000          0.000000    0.000000   
     2    0.459875      0.459875        -71.750492    0.459875   
     3    0.000000      0.000000          0.000000    0.000000   
1    0    0.000000      0.000000          0.000000    0.000000   
     1    0.000000      0.000000          0.000000    0.000000   
     2    0.000000      0.000000          0.000000    0.000000   
     3    0.000000      0.000000          0.000000    0.000000   
2    0    0.000000      0.000000          0.000000    0.000000   
     1    0.000000      0.000000          0.000000    0.000000   
     2    0.000000      0.000000          0.000000    0.000000   
     3    0.463005      0.463005        108.072321    0.463005   

          ikss_to_degree  p_from_mw  q_from_mvar   p_to_mw  q_to_mvar  \
line bus                                                                
0    0          0.000000   0.000000    -0.000000  0.000000  -0.000000   
     1          0.000000   0.000000    -0.000000  0.000000  -0.000000   
     2        108.249508   2.090529     2.848703 -0.634455  -1.268910   
     3          0.000000   0.000000    -0.000000  0.000000  -0.000000   
1    0          0.000000   0.000000    -0.000000  0.000000  -0.000000   
     1          0.000000   0.000000    -0.000000  0.000000  -0.000000   
     2          0.000000   0.000000    -0.000000  0.000000  -0.000000   
     3          0.000000   0.000000    -0.000000  0.000000  -0.000000   
2    0          0.000000   0.000000    -0.000000  0.000000  -0.000000   
     1          0.000000   0.000000    -0.000000  0.000000  -0.000000   
     2          0.000000   0.000000    -0.000000  0.000000  -0.000000   
     3        -71.927679  -0.643122    -1.286244  1.627099   2.353827   

          vm_from_pu  va_from_degree  vm_to_pu  va_to_degree     ip_ka  \
line bus                                                                 
0    0      0.004601       -7.244659  0.004601     -7.244659  0.000000   
     1      0.016526       -8.854432  0.016526     -8.854432  0.000000   
     2      0.040328      -18.023687  0.016192     -8.315543  0.900379   
     3      0.032438      -16.582110  0.032438    -16.582110  0.000000   
1    0      0.004601       -7.244659  0.004601     -7.244659  0.000000   
     1      0.016526       -8.854432  0.016526     -8.854432  0.000000   
     2      0.016192       -8.315543  0.016192     -8.315543  0.000000   
     3      0.032438      -16.582110  0.032438    -16.582110  0.000000   
2    0      0.004601       -7.244659  0.004601     -7.244659  0.000000   
     1      0.016526       -8.854432  0.016526     -8.854432  0.000000   
     2      0.040328      -18.023687  0.040328    -18.023687  0.000000   
     3      0.016302       -8.492730  0.032438    -16.582110  0.908972   

            ith_ka  
line bus            
0    0    0.000000  
     1    0.000000  
     2    0.462274  
     3    0.000000  
1    0    0.000000  
     1    0.000000  
     2    0.000000  
     3    0.000000  
2    0    0.000000  
     1    0.000000  
     2    0.000000  
     3    0.465446

The results above show the line results for the short-circuit calculation. The result table has a MultiIndex, which means that the results are shown for every line ("line" index column) and for every situation defined by a different fault location bus separately (index column "bus"). Note that not only the current magnitude values are available, but also the current angles, along with active and reactive power flows, and voltage magnitude and angle.

## Asymetrical Two-Phase Short-Circuit Calculation

All calculations above can be carried out for a two-phase short-circuit current in the same way by specifying "2ph" in the fault parameter:

In [20]:
net = ring_network()
sc.calc_sc(net, fault="2ph", ip=True, ith=True)
net.res_bus_sc

c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:240: RuntimeWarning: invalid value encountered in log
  m = (np.exp(4 * f * tk_s * np.log(kappa - 1)) - 1) / (2 * f * tk_s * np.log(kappa - 1))


,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.227273,28.867513,0.438065,0.228391,175.878566,502.510189
1,0.412621,26.204958,0.816306,0.414860,44.276323,139.778739
2,0.404149,25.666902,0.792775,0.406272,46.571323,142.268739
3,0.406938,25.844045,0.800468,0.409099,45.806323,141.438739


Two phase short-circuits are often used for minimum short-circuit calculations:

In [21]:
net = ring_network()
net.line["endtemp_degree"] = 150
sc.calc_sc(net, fault="2ph", case="min", ip=True, ith=True)
net.res_bus_sc

c:\users\rbolgaryn\repos\pandapower\pandapower\shortcircuit\currents.py:240: RuntimeWarning: invalid value encountered in log
  m = (np.exp(4 * f * tk_s * np.log(kappa - 1)) - 1) / (2 * f * tk_s * np.log(kappa - 1))


,ikss_ka,skss_mw,ip_ka,ith_ka,rk_ohm,xk_ohm
0,0.181818,23.094011,0.400566,0.183369,118.650262,593.251309
1,0.332920,21.143233,0.745538,0.335995,29.969247,162.464019
2,0.326772,20.752827,0.717815,0.329521,33.457647,164.954019
3,0.328807,20.882073,0.726834,0.331657,32.294847,164.124019


## Single Phase Short-Circuit Calculation


pandapower can also calculate single phase short-circuits. The ground fault however depends on the zero-sequence parameters of the network, which have to be added in order to calculate a single line to ground fault:

In [22]:
net = ring_network()

#r/x ratio in zero sequence parameters
net.ext_grid["r0x0_max"] = 0.4
net.ext_grid["x0x_max"] = 1.0

#zero sequence line parameters
net.line["r0_ohm_per_km"] = 0.244
net.line["x0_ohm_per_km"] = 0.336
net.line["c0_nf_per_km"] = 2000

#transformer vector group, zero sequence short circuit voltage
#and zero sequence magnetizing impedance
net.trafo["vector_group"] = "Dyn"
net.trafo["vk0_percent"] = 5.
net.trafo["vkr0_percent"] = 0.4
net.trafo["mag0_percent"] = 10
net.trafo["mag0_rx"] = 0.4
net.trafo["si0_hv_partial"] = 0.9

In [23]:
sc.calc_sc(net, fault="1ph")

In [24]:
net.res_bus_sc

,ikss_ka,rk0_ohm,xk0_ohm,rk_ohm,xk_ohm
0,0.261047,201.004076,502.510189,175.878566,502.510189
1,0.698664,0.673006,6.835257,44.276323,139.778739
2,0.668867,5.656564,12.811454,46.571323,142.268739
3,0.680010,3.287676,10.346508,45.806323,141.438739
